# Case 3. Patient Drug Review
**Neural Networks for Machine Learning Applications**<br>
27.02.2023<br>
Erik Holopainen, Alejandro Rosales Rodriguez and Brian van den Berg<br>
[Information Technology, Bachelor's Degree](https://www.metropolia.fi/en/academics/bachelors-degrees/information-technology)<br>
[Metropolia University of Applied Sciences](https://www.metropolia.fi/en)

## 1. Introduction

Instructions: Write here why this Notebook was created, what were the main objectives.

## 2. Setup

Instructions: Write here shortly what libraries were used and why.

In [9]:
import pandas as pd
import os

## 3. Dataset

Instructions: Describe here brielfy the data and its main characteristics. Remember document the code.  

In [36]:
# Define the input variables
inputDir = 'input'
inputPaths = []

# Get the .csv files in the input folder
for file in os.listdir(inputDir):
    if file.endswith('.csv'):
        inputPaths.append(os.path.join(inputDir, file))

# Print the input paths
print(inputPaths)

# Define the dataframe
df = pd.DataFrame()

# Append all the input files
for path in inputPaths:
    df = pd.concat([df, pd.read_csv(path)], ignore_index=True)

# Drop the unique id column
df = df.drop(['uniqueID'], axis=1)

# Shuffle the dataframe
df = df.sample(frac=1)
df = df.reset_index(drop=True)

# Display the dataframe
display(df)

# Display the dataframe description
print("Description of the dataframe:")
display(df.describe().T)

['input\\drugsComTest_raw.csv', 'input\\drugsComTrain_raw.csv']


,drugName,condition,review,rating,date,usefulCount
0,Asenapine,Bipolar Disorde,"""I have just started taking Saphris 5 mg in th...",4,17-Dec-11,7
1,Amphetamine / dextroamphetamine,ADHD,"""I hate the loss of appetite, other than that,...",7,31-Jul-10,8
2,Zanaflex,Muscle Spasm,"""I first was given Zanaflex about 14 years ago...",10,30-Mar-15,120
3,Levonorgestrel,Abnormal Uterine Bleeding,"""Got my IUD inserted toward the end of my peri...",1,26-Jul-15,7
4,Effexor,Depression,"""Hi,\r\r\n\r\r\nI have been using Effexor xr f...",1,26-Jul-16,15
...,...,...,...,...,...,...
215058,Treprostinil,Pulmonary Hypertension,"""Been on Tyvaso for 3 years. It has improved m...",8,22-Dec-16,0
215059,Mirtazapine,Anxiety,"""I began taking this medicine about 1 year ago...",5,5-Sep-15,23
215060,Effexor XR,Social Anxiety Disorde,"""During menstruation, I felt like I was out of...",2,17-Jan-16,13
215061,Lo Loestrin Fe,Birth Control,"""In the first few months that I was on this dr...",7,3-Jan-17,5


Description of the dataframe:


,count,mean,std,min,25%,50%,75%,max
rating,215063.0,6.990008,3.275554,1.0,5.0,8.0,10.0,10.0
usefulCount,215063.0,28.001004,36.346069,0.0,6.0,16.0,36.0,1291.0


## 4. Preprocessing

Instructions: Describe:

- how the missing values are handled
- conversion of textual and categorical data into numerical values (if needed)
- how the data is splitted into train, validation and test sets
- the features (=input) and labels (=output), and 
- how the features are normalized or scaled

In [11]:
# Your code, split into several cells for clarity

## 5. Modeling

Instructions: Write a short description of the model: 

- selected loss, optimizer and metrics settings, and 
- the summary of the selected model architecture. 

In [12]:
# Your code, use several cells, if necessary for the clarity

## 6. Training

Instructions: Write a short description of the training process, and document the code for training and the total time spend on it. 

In [13]:
# Your code

## 7. Performance and evaluation

Instructions: 

- Show the training and validation loss and accuracy plots
- Interpret the loss and accuracy plots (e.g. is there under- or over-fitting)
- Describe the final performance of the model with test set 

In [14]:
# Your code

## 8. Discussion and conclusions

Instructions: Write

- What settings and models were tested before the best model was found
    - What where the results of these experiments 
- Summary of  
    - What was your best model and its settings 
    - What was the final achieved performance 
- What are your main observations and learning points
- Discussion how the model could be improved in future 

**Note:** Remember to evaluate the final metrics using the test set. 
